In [135]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Lambda
from keras.layers import LSTM, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import tensorflow as tf

In [103]:
df = pd.read_csv('../../data/scheme-progs.csv')
df = df.sample(frac=1).reset_index(drop=True)
df['noerr'] = 1 - df['error']

In [137]:
train_x = df.iloc[:38000,0:1].values[:,0]
train_y = df.iloc[:38000,2:].values
test_x = df.iloc[38000:,0:1].values[:,0]
test_y = df.iloc[38000:,2:].values

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(38000,) (38000, 2) (9890,) (9890, 2)


In [138]:
SYMBOLS = ['(', ')', ' ', 'x', 'y', '1', '2', '0', '-', 'λ', '+', '-', '*', '/']
N_SYMBOLS = len(SYMBOLS)
TOKENS = dict((c, i) for i, c in enumerate(SYMBOLS))
MAX_LEN = max(list(map(len, (list(train_x)+list(test_x)))))

def tokenize_string(s):
    ret = np.zeros((MAX_LEN, N_SYMBOLS), dtype=bool)
    for i, char in enumerate(s):
        ret[i, TOKENS[char]] = 1
    return ret

def tokenize(a):
#     ret = np.zeros(len(a), MAX_LEN, N_SYMBOLS)
#     sentences = 
#     for i, sentence in enumerate(s):
#         for t, char in enumerate(sentence):
    return np.array(list(map(lambda s: tokenize_string(s), list(a))))

In [139]:
train_x = tokenize(train_x)
test_x = tokenize(test_x)

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(38000, 35, 14) (38000, 2) (9890, 35, 14) (9890, 2)


In [141]:
model = Sequential()
model.add(LSTM(128, input_shape=(MAX_LEN,N_SYMBOLS)))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.fit(train_x,
          train_y,
          batch_size=500,
          epochs=10,
          validation_split=0.3,
          callbacks=[EarlyStopping(patience=2)])

Train on 26600 samples, validate on 11400 samples
Epoch 1/10
26600/26600 [==============================] - 28s - loss: 0.5272 - acc: 0.7815 - val_loss: 0.4090 - val_acc: 0.7818
Epoch 2/10
26600/26600 [==============================] - 27s - loss: 0.3245 - acc: 0.8545 - val_loss: 0.2756 - val_acc: 0.8647
Epoch 3/10
26600/26600 [==============================] - 27s - loss: 0.2525 - acc: 0.8757 - val_loss: 0.2060 - val_acc: 0.9011
Epoch 4/10
26600/26600 [==============================] - 27s - loss: 0.2535 - acc: 0.8685 - val_loss: 0.2176 - val_acc: 0.8801
Epoch 5/10
26600/26600 [==============================] - 27s - loss: 0.2056 - acc: 0.8991 - val_loss: 0.1803 - val_acc: 0.9267
Epoch 6/10
26600/26600 [==============================] - 27s - loss: 0.1821 - acc: 0.9182 - val_loss: 0.1499 - val_acc: 0.9446
Epoch 7/10
26600/26600 [==============================] - 27s - loss: 0.1428 - acc: 0.9444 - val_loss: 0.1292 - val_acc: 0.9508
Epoch 8/10
26600/26600 [==============================

In [142]:
model.evaluate(x=test_x, y=test_y, batch_size=500)

9890/9890 [==============================] - 2s     


[0.083157962235208716, 0.96855409875798393]

In [179]:
tests = tokenize(np.array(['(/ 1 1)', '(/ 1 ()())))))((((0)']))

In [180]:
model.predict(tests)

array([[ 0.01850473,  0.9814952 ],
       [ 0.17510785,  0.82489216]], dtype=float32)